# Pre-processing of public interview data

Since planing to use OpenAI, working with the publicly uploaded interview data available from the [Arctic Data Center Catalog](https://arcticdata.io/catalog).

We will specifically use the following PDFs of compiled interviews:   
* [Interviews with Water Sector Stakeholders in Rural Alaska, 2021-2022](https://arcticdata.io/catalog/view/doi%3A10.18739%2FA29Z90D4B)
* [Interview Data from April 2022: End-Users' Perceptions of Water Services in Rural Alaska](https://arcticdata.io/catalog/view/doi%3A10.18739%2FA26Q1SJ48)

## Libraries & Configurations

In [1]:
# Load OpenAI API key from local .env file
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
import fitz # this is PyMuPDF. Weird name because developers.
import re

import pandas as pd 
import json
import numpy as np

## Processing Functions

In [3]:
# Font Flag descriptions from PyMuPDF documentation 
# https://pymupdf.readthedocs.io/en/latest/recipes-text.html#how-to-analyze-font-characteristics
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

In [67]:
def load_and_chunk(pdffile):
    doc = fitz.open(os.path.join(pdffile)) # Use the 'Blocks' option in the PyMuPDF package
    
    # Load each page in doc into a dictionary
    block_dict = {}
    page_num = 1
    for page in doc:
        # Load blocks for each page
        block_dict[page_num] = page.get_text('dict')['blocks']
        page_num += 1
    
    # Read blocks into dataframe
    blocksdf = pd.DataFrame()
    for page_num, blocks in block_dict.items():
        blockdf = pd.DataFrame(blocks)
        blockdf['page_num'] = page_num

        if blocksdf.empty:
            blocksdf = blockdf
        else:
            blocksdf = pd.concat([blocksdf, blockdf])
            
    # Extract span information for each block
    textcols = ['page_num', 'number', 'lines']
    textdf = blocksdf[blocksdf['type'] == 0][textcols].reset_index(drop=True)
    textdf = textdf.explode('lines')
    textdf['spans'] = textdf['lines'].apply(lambda x: x['spans'])
    textdf = textdf.explode('spans')
    textdf.reset_index(inplace=True, drop=True)           
    
    # Turn spans dictionary into dataframe and then merge back in
    spans_df = pd.DataFrame(textdf['spans'].tolist())
    alldf = pd.concat([textdf, spans_df], axis=1)    
    
    # Subset data and rename
    col_dict = {
        'page_num' : 'Page',
        'number': 'Block',
        'size': 'font_size',
        'flags':'font_flag', 
        'font': 'font',
        'color': 'color',
        'bbox':'bbox',
        'text':'text'}
    df = alldf[col_dict.keys()].copy()
    df.rename(columns=col_dict, inplace=True)    
    
    # Drop lines of empty text
    dftext = df[(df.text != ' ') & (df.text != '  ')].copy()

    dftext['page_text'] = dftext.apply(lambda x: str(x['Page'])+ ' ' == x['text'], axis = 1)
    dftext = dftext[~dftext['page_text']].copy()

    dftext.reset_index(drop=True, inplace=True)    
    
    # Add columnst o flag if text is lower case, upper case, convert Flag to human readable form
    dftext['is_bold'] = dftext['font'].apply(lambda x: True if "bold" in x.lower() else False)
    dftext['is_upper'] = dftext['text'].apply(lambda x: True if re.sub("[\(\[].*?[\)\]]", "", x).isupper() else False)
    dftext['Flag'] = dftext['font_flag'].apply(lambda x: flags_decomposer(x))     
    
    return dftext

## Load PDF and Chunk

### Interviews with Water Sector Stakeholders in Rural Alaska, 2021-2022
Note: the formatting of the interviews changes between Interview 14 and 15 / page 250 and 251 in the pdf

In [50]:
# Pointer to local copy of the pdf of interview data
pdffile = '../DATA/Arctic_Data_21_22/data/2023_08_22_Compiled.pdf'

In [68]:
int_21_22 = load_and_chunk(pdffile)

In [66]:
int_21_22.head(2)

,Page,Block,font_size,font_flag,font,color,bbox,text,page_text,is_bold,is_upper,Flag,time_marker,Interview
0,1,1,11.04,16,Calibri-Bold,0,"(276.04998779296875, 74.18002319335938, 338.56...",INTERVIEW 1,False,True,True,"sans, proportional, bold",False,INTERVIEW 1
1,1,2,11.04,16,Calibri-Bold,0,"(72.02400207519531, 96.62002563476562, 137.515...",Interviewee 1,False,True,False,"sans, proportional, bold",False,INTERVIEW 1


### Interview Data from April 2022: End-Users' Perceptions of Water Services in Rural Alaska

In [32]:
# Pointer to local copy of the pdf of interview data
yk_pdffile = '../DATA/YK_Delta_April22/data/YKDelta_End_User_Interviews_Anonymized.pdf'

In [303]:
apr_22 = load_and_chunk(yk_pdffile)

In [35]:
apr_22.head(2)

,Page,Block,font_size,font_flag,font,color,bbox,text,page_text,is_bold,is_upper,Flag
0,1,0,11.04,0,ArialMT,9605778,"(54.0, 741.9720458984375, 377.53912353515625, ...",Transcript anonymized according to IRB for pri...,False,False,False,"sans, proportional"
1,1,0,11.04,0,ArialMT,0,"(540.7000122070312, 745.572021484375, 563.3291...",- 1 -,False,False,False,"sans, proportional"


## Process text data into standardized data format with: Interview, Page, Block, Speaker, Time (if available), Text

### Interviews with Water Sector Stakeholders in Rural Alaska, 2021-2022
Split text at page 250 where structure of Interview documents changes.  
**Assumption**: this reflects the split between the 2 interview sessions (Nov 21 and August 22), and we will preserve that distinction for possible use later

### Nov 21 Interviews

In [212]:
nov21 = int_21_22[int_21_22.Page <= 250].copy()

In [213]:
# Flag Time marks in interview, which are distinguished logically by the color of the font
nov21['time_marker'] = nov21['color'].apply(lambda x: True if x == 6123908 else False)

In [214]:
# The title of the section (which defines which Interview the text came from) is distinguished by is_upper = True 
# Fill forward for all rows until the next Interview begins
nov21['Interview'] = nov21.apply(lambda x: x['text'] if x['is_upper'] and x['is_bold'] else np.NaN, axis = 1)
nov21['Interview'] = nov21['Interview'].fillna(method='ffill')

In [215]:
# Remove text line that delineates the Interview
nov21 = nov21[nov21['text'] != nov21['Interview']]

In [216]:
# is_bold = defines speaker, forward fill
nov21['Speaker'] = nov21.apply(lambda x: x['text'] if x['is_bold'] else np.NaN, axis = 1)
nov21['Speaker'] = nov21['Speaker'].fillna(method='ffill')

In [217]:
# Get the shifted text value and use this as the time marker for the speaker value, then forward fill
nov21['shift'] = nov21['text'].shift(-1)
nov21['Time_Mark'] = nov21.apply(lambda x: x['shift'] if x['is_bold'] else np.NaN, axis = 1)
nov21['Time_Mark'] = nov21['Time_Mark'].fillna(method='ffill')

In [218]:
# Drop speaker and time marker rows
nov21 = nov21[(~nov21['is_bold']) & (~nov21['time_marker'])].copy()
nov21.reset_index(drop=True, inplace=True)

In [219]:
# rename 'text' --> 'Text' for consistency
nov21.rename(columns={"text": "Text"}, inplace=True)

In [221]:
# Subset datafrane with specific columns for analysis
keep_cols = ['Interview','Page', 'Block', 'Speaker', 'Time_Mark']
nov21_b = nov21[keep_cols + ['Text']].copy()

#### Group all text from particular Speaker at one point into single row
This is necessary because the pdf reader will split text into separate blocks randomly in the middle of a sentence.  in downsrteam steps can always resplit into sentences.

In [292]:
# Aggregate all text from a specific speaker
nov21_grouped = nov21_b.groupby(keep_cols).agg({'Text': lambda x: ' '.join(x)})
nov21_grouped.reset_index(inplace=True)
nov21_grouped.sort_values(by=['Interview','Time_Mark'],inplace=True)

In [293]:
nov21_grouped.head(3)

,Interview,Page,Block,Speaker,Time_Mark,Text
0,INTERVIEW 1,1,2,Interviewee 1,00:00,"A sewer project called the [REDACTED], I forge..."
2,INTERVIEW 1,1,2,Interviewer 1,02:56,So like with the new subdivisions coming online?
1,INTERVIEW 1,1,2,Interviewee 1,02:59,"That's what I'm worried about. Yeah, so we're ..."


### Aug 22 Interviews

In [226]:
aug22 = int_21_22[int_21_22.Page > 250].copy()

In [227]:
# The title of the section (which defines which Interview the text came from) is distinguished by is_upper = True 
# Fill forward for all rows until the next Interview begins
aug22['Interview'] = aug22.apply(lambda x: x['text'] if x['is_upper'] and x['is_bold'] else np.NaN, axis = 1)
aug22['Interview'] = aug22['Interview'].fillna(method='ffill')
# Remove text line that delineates the Interview
aug22 = aug22[aug22['text'] != aug22['Interview']]

In [228]:
# Use bbox to define if text includes speaker or starts with speech
aug22['left_col_present'] = aug22.apply(lambda x: True if x['bbox'][0] < 73 else False, axis = 1)

In [231]:
# PDF sometimes keeps both cols together and sometimes not.  Left col present in bbox indicates the text field contains the speaker inforamtion.  
# May also include text, but this is uncertain.  If the bbox starts at higher x this is just text without speaker
aug22_left_col = aug22[aug22['left_col_present']].copy()
aug22_right_col = aug22[~aug22['left_col_present']].copy()

In [232]:
# if left column present, split on first instance of :
aug22_left_col[['Speaker','Text']] = aug22_left_col['text'].str.split(':',n=1,expand=True)

In [233]:
aug22_right_col['Speaker'] = np.nan

In [234]:
aug22_right_col['Text'] = aug22_right_col['text'] 

In [294]:
aug22_merged = pd.concat([aug22_left_col, aug22_right_col])

In [295]:
# Sort back into original order
aug22_merged.sort_index(inplace=True)

In [296]:
# Some text splits into single row, others are spread across rows due to pdf read in.  
# Forward fill the speaker into rows with speech text
aug22_merged['Speaker'] = aug22_merged['Speaker'].fillna(method='ffill')

In [297]:
# Drop lines of empty text
aug22_merged = aug22_merged[(aug22_merged.Text != ' ') & (aug22_merged.Text != '  ')].copy()

In [298]:
aug22_merged['Time_Mark'] = aug22_merged.groupby(['Interview']).cumcount()+1

In [299]:
# Subset datafrane with specific columns for analysis
keep_cols = ['Interview','Page', 'Block', 'Speaker','Time_Mark']
aug22_merged = aug22_merged[keep_cols + ['Text']].copy()

#### Group all text from particular Speaker at one point into single row

This is necessary because the pdf reader will split text into separate blocks randomly in the middle of a sentence. in downsrteam steps can always resplit into sentences.


In [300]:
# Aggregate all text from a specific speaker
aug22_merged = aug22_merged.groupby(keep_cols).agg({'Text': lambda x: ' '.join(x)})
aug22_merged.reset_index(inplace=True)
aug22_merged.sort_values(by=['Interview','Time_Mark'],inplace=True)

### April 22 Interviews (YK Delta)

In [304]:
# Expand bbox for use in cleanup
apr_22[['x0','y0','x1','y1']] = pd.DataFrame(apr_22['bbox'].tolist(), index= apr_22.index)

In [305]:
# Drop rows for footer page number and transcript anonymization text. use logic bbox y0 > 740
apr_22 = apr_22[apr_22['y0'] < 740].copy()

In [306]:
# use is_bold as flag for Interview label
# Fill forward for all rows until the next Interview begins
apr_22['Interview'] = apr_22.apply(lambda x: x['text'] if x['is_bold'] else np.NaN, axis = 1)
apr_22['Interview'] = apr_22['Interview'].fillna(method='ffill')
# Remove text line that delineates the Interview
apr_22 = apr_22[apr_22['text'] != apr_22['Interview']]

In [307]:
# Font color 6123908 indicates Speaker
apr_22['Speaker'] = apr_22.apply(lambda x: x['text'] if x['color']==6123908 else np.NaN, axis = 1)
apr_22['Speaker'] = apr_22['Speaker'].fillna(method='ffill')

In [308]:
# Drop the columns where Speaker = Text
apr_22 = apr_22[~(apr_22['text']==apr_22['Speaker'])]

In [309]:
# rename 'text' --> 'Text' for consistency
apr_22.rename(columns={"text": "Text"}, inplace=True)

In [310]:
# Add cumcount column as time_mark since no times given in interview transcript, so this explicitly preserves speech order
apr_22['Time_Mark'] = apr_22.groupby(['Interview']).cumcount()+1

In [311]:
# Subset datafrane with specific columns for analysis
keep_cols = ['Interview','Page', 'Block', 'Speaker', 'Time_Mark']
apr_22 = apr_22[keep_cols + ['Text']].copy()

#### Group all text from particular Speaker at one point into single row
This is necessary because the pdf reader will split text into separate blocks randomly in the middle of a sentence.  in downsrteam steps can always resplit into sentences.

In [312]:
# Aggregate all text from a specific speaker
apr_22 = apr_22.groupby(keep_cols).agg({'Text': lambda x: ' '.join(x)})
apr_22.reset_index(inplace=True)
apr_22.sort_values(by=['Interview','Time_Mark'],inplace=True)

In [313]:
apr_22.head()

,Interview,Page,Block,Speaker,Time_Mark,Text
16,Interview 1,1,2,Interviewer,1,And we're good to go. So.
0,Interview 1,1,2,Interviewee,2,"So we use the water, it's just like back at ho..."
1,Interview 1,1,2,Interviewee,3,everyday like necessity skills like we use it ...
2,Interview 1,1,2,Interviewee,4,"we use it to like basically basically like, li..."
3,Interview 1,1,2,Interviewee,5,"somewhere around six, six to twelve gallons a ..."


## Combine Data into single dataframe

#### Standardize format and column order

In [314]:
# Assign interviews to a PDF set to link to the catalog reference
nov21_grouped['Set'] = '21-22 Combined'
aug22_merged['Set'] = '21-22 Combined'
apr_22['Set'] = 'April 22'

In [315]:
col_order = ['Set','Interview','Page','Block', 'Speaker', 'Text','Time_Mark']

In [316]:
all_public_interviews = pd.concat([nov21_grouped[col_order], aug22_merged[col_order], apr_22[col_order]])

In [317]:
all_public_interviews.to_csv('all_public_interviews.csv')

In [318]:
all_public_interviews.head()

,Set,Interview,Page,Block,Speaker,Text,Time_Mark
0,21-22 Combined,INTERVIEW 1,1,2,Interviewee 1,"A sewer project called the [REDACTED], I forge...",00:00
2,21-22 Combined,INTERVIEW 1,1,2,Interviewer 1,So like with the new subdivisions coming online?,02:56
1,21-22 Combined,INTERVIEW 1,1,2,Interviewee 1,"That's what I'm worried about. Yeah, so we're ...",02:59
3,21-22 Combined,INTERVIEW 1,1,2,Interviewer 1,I think the scope of what we're trying to look...,04:13
4,21-22 Combined,INTERVIEW 1,2,1,Interviewee 1,One challenge on the hauled side of it for us....,04:47
